# Final Project

General Plan:
- Make regression/model 
- Feed in time, latitude, longitude to predict weather at that location at that time
- Possibly create combination variables to add extra predictors
- Plot these weather events on a map 

## Part 0: Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Part 1: Data Management/Representation

In [3]:
# import the data from csv 
weather_data = pd.read_csv('weather_data.csv')

# display data
weather_data

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7479160,W-7577193,Storm,Severe,2021-12-26 18:19:00,2021-12-26 18:29:00,0.00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
7479161,W-7577194,Snow,Light,2021-12-26 18:29:00,2021-12-26 18:38:00,0.02,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
7479162,W-7577195,Snow,Light,2021-12-28 00:53:00,2021-12-28 02:27:00,0.08,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
7479163,W-7577196,Snow,Moderate,2021-12-28 02:27:00,2021-12-28 03:09:00,0.09,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0


## Exploratory Data Analysis

## Hypothesis testing

## Communication of Insights Attained